In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import torch

In [6]:
import torch
import torch.onnx
from torchvision import models

# Load the model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Linear(model.fc.in_features, 7)
state_dict = torch.load('..\\models\\resnet18_weighted.pth', weights_only=True)
model.load_state_dict(state_dict)

# Define a dummy input tensor with the same shape as the input data
dummy_input = torch.randn(1, 3, 100, 100)  # Adjust the shape according to your model's input

# Export the model to ONNX format
onnx_path = "../models/resnet18_v2.onnx"
torch.onnx.export(model, dummy_input, onnx_path, 
                  export_params=True, opset_version=10, 
                  do_constant_folding=True, 
                  input_names=['input'], output_names=['output'])

print(f"Model has been converted to ONNX and saved at {onnx_path}")

Model has been converted to ONNX and saved at ../models/resnet18_v2.onnx


In [85]:
# Load the ONNX model
import onnx
onnx_path = "../models/resnet18.onnx"
model = onnx.load(onnx_path)

In [86]:
from PIL import Image
import numpy as np

img = Image.open(r'C:\Users\Faruq\Desktop\DataScience\Projects\emotion-analysis\rafdb_data\test\6\test_0228_aligned.jpg')
img = img.resize((100, 100))
img = np.array(img)
img = img / 255.0
img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)
img = img.astype(np.float32)
img.shape

# Inference
model = onnx.load(onnx_path)
onnx.checker.check_model(model)
import onnxruntime as rt

ort_session = rt.InferenceSession(onnx_path)
input_name = ort_session.get_inputs()[0].name
ort_inputs = {input_name: img}
outputs = ort_session.run(None, ort_inputs)

probs = np.exp(outputs[0]) / np.sum(np.exp(outputs[0]))
print(np.argmax(probs), probs)

6 [[0.02793481 0.00141386 0.00485399 0.03123341 0.05615688 0.00480732
  0.8735997 ]]


In [ ]:
label_dict = {'0': 'Surprise', '1': 'Fear', '2': 'Disgust', '3': 'Happy', '4': 'Sadness', '5': 'Anger', '6': 'Neutral'}